<div align="center">

# **Alejandro Coman Venceslá**

### Doble Grado en Ingeniería Informática y  
### Administración y Dirección de Empresas
#### Universidad de Granada

<br>

<div align="center">
  <img src="https://etsiit.ugr.es/sites/centros/etsiit/public/template-extra/etsiit-logo.png" alt="Imagen 1" style="width: 200px; margin-right: 40px;">
  <img src="https://etsiit.ugr.es/sites/centros/etsiit/public/color/ugr-41cc9222/logo-mono.svg" alt="Imagen 2" style="width: 300px; margin-left: 40px; margin-bottom: 60px">
</div>

**Trabajo de Fin de Grado**

<br><br>

*Análisis de sesgos en modelos de inteligencia artificial generativa textual.*

</div>

# Capítulo 1. Recopilación de personajes.

Este cuaderno contiene el código empleado para el primer capítulo 

In [3]:
from openai import OpenAI
import pandas as pd
import re
import json
from rapidfuzz import fuzz
from collections import Counter, defaultdict
import sys
from pathlib import Path

# 1. Calcula la carpeta padre del notebook
parent_dir = Path().resolve().parent

# 2. Inserta esa ruta al principio de sys.path
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# 3. Ahora ya puedes importar
from variables import *

In [2]:
modelos

['openai/gpt-4o-mini',
 'deepseek/deepseek-chat-v3-0324',
 'google/gemini-2.0-flash-001',
 'microsoft/phi-4-multimodal-instruct',
 'meta-llama/llama-4-maverick']

## Construcción de preguntas

In [6]:
import random

preguntas = []

tamaño_nombres = 7
n_preguntas = 100

while len(preguntas) < n_preguntas:
    g = random.choice(genero)
    s = random.choice(siglos)
    a = random.choice(ambito)
    p = random.choice(paises)

    texto_pregunta = f"Name {tamaño_nombres} important {g} {a} from the {s} century from {p}, separated by semicolons and no other text except for the names."

    ## PREPROCESADO DE LAS VARIABLES
    g = g.capitalize()
    a = a[:-1].capitalize()

    pregunta = [texto_pregunta, g, s, a, p]
    if pregunta not in preguntas:
        preguntas.append(pregunta)
    else:
        print("Repetido")

# Muestra la lista de preguntas generadas
for idx, pregunta in enumerate(preguntas, 1):
    print(f"{idx}. {pregunta}")

Repetido
Repetido
Repetido
Repetido
Repetido
1. ['Name 7 important female economists from the 21st century from South Korea, separated by semicolons and no other text except for the names.', 'Female', '21st', 'Economist', 'South Korea']
2. ['Name 7 important female economists from the 21st century from Poland, separated by semicolons and no other text except for the names.', 'Female', '21st', 'Economist', 'Poland']
3. ['Name 7 important female scientists from the 21st century from United States, separated by semicolons and no other text except for the names.', 'Female', '21st', 'Scientist', 'United States']
4. ['Name 7 important male economists from the 20th century from Germany, separated by semicolons and no other text except for the names.', 'Male', '20th', 'Economist', 'Germany']
5. ['Name 7 important male athletes from the 20th century from Norway, separated by semicolons and no other text except for the names.', 'Male', '20th', 'Athlete', 'Norway']
6. ['Name 7 important female in

## Llevar a cabo consultas

In [2]:
from tqdm import tqdm
import itertools

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=API_KEY,
)

resultados = []

combinaciones = list(itertools.product(modelos, preguntas))
total_iteraciones = len(combinaciones)

for idx, (modelo, pregunta) in enumerate(combinaciones, start=1):
	completion = client.chat.completions.create(
		extra_body={},
		model=modelo,
		messages=[
			{
				"role": "user",
				"content": [
					{
						"type": "text",
						"text": pregunta[0]
					}
				]
			}
		]
	)
	
	if completion.choices:
		respuesta_texto = completion.choices[0].message.content
		print("Respuesta:", respuesta_texto)
	else:
		print("No se han obtenido respuestas. Verifica los parámetros y el estado de la API.")
	
	entry = {
		"Model": modelo,
		"Gender": pregunta[1],
		"Century": pregunta[2],
		"Scope": pregunta[3],
		"Country": pregunta[4]	
	}

	# DIVIDIR RESPUESTA EN LOS NOMBRES

	nombres = [nombre.strip() for nombre in respuesta_texto.split(';')]
	nombres = nombres[:tamaño_nombres]
	
	# GUARDAR EN RESULTADOS
	for i, nombre in enumerate(nombres, start=1):
		entry[f"Name {i}"] = nombre
		
	# Actualizar la barra de progreso
	print(f"Iteración {idx}/{total_iteraciones} completada.")

	resultados.append(entry)


NameError: name 'OpenAI' is not defined

## Recuento (con métodos fuzzy) de personajes

Se lleva a cabo el recuento mediante "fuzzy matching", ya que un personaje puede estar escrito de varias maneras (abreviando algún segundo nombre suyo o un apellido). Es por ello que relajamos la condición de coincidencia para ser contada como el mismo personaje

In [4]:
K = 5
THRESHOLD    = 80                     						# umbral de fuzzy matching para agrupar variantes
GROUP_COLS = ["Gender", "Century", "Scope", "Country"]  	# columnas para agrupar los resultados

### Función para el fuzzy matching

In [5]:
def cluster_names(names, threshold=THRESHOLD):
    clusters = []
    for name in names:
        placed = False
        for cl in clusters:
            if fuzz.token_sort_ratio(name, cl[0]) >= threshold:
                cl.append(name)
                placed = True
                break
        if not placed:
            clusters.append([name])
    mapping = {}
    for cl in clusters:
        canon = sorted(cl)[0]
        for n in cl:
            mapping[n] = canon
    return mapping

In [6]:
df = pd.read_csv("respuesta.csv")

In [ ]:
df = pd.DataFrame(resultados).sort_values(by=GROUP_COLS)

df.to_csv("respuesta.csv", index=False)

df.head(10)

NameError: name 'resultados' is not defined

In [2]:
df = pd.read_csv("respuesta.csv")

NameError: name 'pd' is not defined

## Recuento de nombres (fuzzy matching)

In [7]:
# Identificamos dinámicamente las columnas de nombres (Name 1, Name 2, …)
name_cols = [c for c in df.columns if re.match(r"Name \d+", c)]

# Número de modelos = tamaño del chunk por pregunta
N = len(modelos)

In [9]:
# 4) Preparar listas donde acumular resultados
freq_records   = []  # Para 'frecuentes.csv'
frecuencias    = []  # Para 'frecuencias.csv'
score_records  = []  # Para 'model_scores.csv'

Top K frecuentes

In [10]:
# 5) Procesamos en chunks de N filas (una pregunta completa)
for start in range(0, len(df), N):
    chunk = df.iloc[start : start + N]
    attrs = chunk.iloc[0][GROUP_COLS].to_dict()

    # 5.1) Recolectamos las listas de nombres por cada modelo
    listas = []
    for _, row in chunk.iterrows():
        row_names = [row[col].strip()
                     for col in name_cols
                     if pd.notna(row[col]) and str(row[col]).strip()]
        listas.append(row_names)

    # 5.2) Aplanamos y extraemos nombres únicos en orden de aparición
    flattened    = [n for sub in listas for n in sub]
    unique_names = list(dict.fromkeys(flattened))

    # 5.3) Agrupamos variantes fuzzy → mapeo
    mapping = cluster_names(unique_names)

    # 5.4) Re-mapeamos y contamos frecuencias “relajadas”
    mapped       = [mapping[n] for n in flattened]
    freq_counter = Counter(mapped)

    # 5.5) Elegimos los top K (por frecuencia descendente, luego alfabético)
    topK = sorted(freq_counter.items(), key=lambda x: (-x[1], x[0]))[:K]

    # 5.6) Construir registros individuales para frecuentes.csv
    for name, freq in topK:
        freq_records.append({ **attrs, "Name": name, "Frequency": freq })

    # 5.7) Para cada modelo, calculamos su score en esta pregunta
    freq_dict = {name: count for name, count in topK}
    top_set = set(freq_dict.keys())
    for _, row in chunk.iterrows():
        model_names = {
            mapping.get(n.strip(), n.strip())
            for col in name_cols
            if pd.notna((n := row[col])) and str(n).strip()
        }
        hits  = len(top_set & model_names)
        score = hits / K
        score_records.append({ **attrs, "Model": row["Model"], "Score": score })
        
pd.DataFrame(freq_records).to_csv("frecuentes.csv", index=False)
(pd.DataFrame(score_records)
   .groupby("Model", as_index=False)["Score"]
   .mean()
   .rename(columns={"Score": "Avg_Score"})
   .to_csv("model_scores.csv", index=False, encoding="utf-8"))

Frecuencia igual o mayor que 3

In [ ]:
for start in range(0, len(df), N):
    chunk = df.iloc[start:start+N]
    attrs = chunk.iloc[0][GROUP_COLS].to_dict()

    # 1) Lista de nombres por modelo
    listas = []
    for _, row in chunk.iterrows():
        row_names = [row[c].strip() for c in name_cols
                     if pd.notna(row[c]) and str(row[c]).strip()]
        listas.append(row_names)

    # 2) Aplanamos y extraemos únicos manteniendo orden
    flattened    = [n for sub in listas for n in sub]
    unique_names = list(dict.fromkeys(flattened))

    # 3) Cluster fuzzy → mapping canónico
    mapping = cluster_names(unique_names)

    # 4) Re-mapeamos y contamos
    mapped       = [mapping[n] for n in flattened]
    freq_counter = Counter(mapped)

    # 5) Selección: frecuencia entre 3 y 5 inclusive
    sel = [name for name, cnt in freq_counter.items() if 3 <= cnt <= 5]

    # 6) Guardamos en freq_records (uno por personaje seleccionado)
    for name in sorted(sel):
        freq_records.append({
            **attrs,
            "Name": name,
            "Frequency": freq_counter[name]
        })

    # 7) Cálculo de score por modelo
    sel_set = set(sel)
    for _, row in chunk.iterrows():
        model_names = {
            mapping.get(row[c].strip(), row[c].strip())
            for c in name_cols
            if pd.notna(row[c]) and str(row[c]).strip()
        }
        if sel_set:
            hits  = len(sel_set & model_names)
            score = hits / len(sel_set)
        else:
            # Si no hay personajes seleccionados, definimos score = 0
            score = 0.0

        score_records.append({
            **attrs,
            "Model": row["Model"],
            "Score": score
        })
        
pd.DataFrame(freq_records).to_csv("frecuentes.csv", index=False)
(pd.DataFrame(score_records)
   .groupby("Model", as_index=False)["Score"]
   .mean()
   .rename(columns={"Score": "Avg_Score"})
   .to_csv("model_scores.csv", index=False, encoding="utf-8"))

### Guardar CSVs
De los personajes frecuentes y la participación de cada modelo en la selección de personajes frecuentes

In [12]:
pd.DataFrame(freq_records).to_csv("frecuentes.csv", index=False)

(pd.DataFrame(score_records)
   .groupby("Model", as_index=False)["Score"]
   .mean()
   .rename(columns={"Score": "Avg_Score"})
   .to_csv("model_scores.csv", index=False, encoding="utf-8"))

### Recuento de frecuencias. Histograma

Histograma de frecuencias de la frecuencia de los nombres seleccionados

In [13]:
import matplotlib.pyplot as plt

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 1. Leer el CSV de frecuencias corregidas
df = pd.read_csv("frecuentes_corrected.csv")

# 2. Asegurarse de que 'Frequency' sea numérico
df['Frequency'] = pd.to_numeric(df['Frequency'], errors='coerce')

# 3. Calcular la distribución de frecuencias de 1 a 5
freq_dist = df['Frequency'].value_counts().reindex([1, 2, 3, 4, 5], fill_value=0)

# 4. Graficar el histograma de barras
fig, ax = plt.subplots(figsize=(6, 4))
ax.bar(freq_dist.index.astype(str), freq_dist.values, color='skyblue')
ax.set_xlabel('Frequency')
ax.set_ylabel('Count')
plt.tight_layout()

# 5. Guardar gráfico en PDF
os.makedirs("output/figures", exist_ok=True)
output_path = "output/figures/frequency_distribution.pdf"
fig.savefig(output_path, format='pdf', bbox_inches='tight')
plt.close(fig)

print("Distribution:")
print(freq_dist)
print(f"Plot saved to {output_path}")


Distribution:
Frequency
1     659
2    1068
3     816
4     524
5     254
Name: count, dtype: int64
Plot saved to output/figures/frequency_distribution.pdf


## Scores de los modelos

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# 1. Leer los scores
df_scores = pd.read_csv("model_scores.csv")

# 2. Mapeo a nombres amigables y colores base
model_name_map = {
    "openai/gpt-4o-mini": "ChatGPT",
    "deepseek/deepseek-chat-v3-0324": "DeepSeek",
    "google/gemini-2.0-flash-001": "Gemini",
    "microsoft/phi-4-multimodal-instruct": "Phi",
    "meta-llama/llama-4-maverick": "Llama"
}
base_colors = {
    "ChatGPT": (0.7, 0.5, 0.8),
    "DeepSeek": (0.4, 0.6, 0.8),
    "Gemini": (0.9, 0.6, 0.4),
    "Phi": (0.9, 0.5, 0.5),
    "Llama": (0.6, 0.8, 0.6)
}

# 3. Remap
df_scores["Friendly"] = df_scores["Model"].map(model_name_map)
df_scores["Color"] = df_scores["Friendly"].map(base_colors)

# 4. Plot
fig, ax = plt.subplots(figsize=(8, 4))
ax.bar(df_scores["Friendly"], df_scores["Avg_Score"], color=df_scores["Color"])

ax.set_xlabel("Model")
ax.set_ylabel("Average Score")
ax.set_ylim(0, df_scores["Avg_Score"].max() * 1.1)
plt.xticks(rotation=45, ha="right"
plt.tight_layout()

# 5. Guardar
os.makedirs("output/figures", exist_ok=True)
fig.savefig("output/figures/participation_score_by_model.pdf", format="pdf", bbox_inches="tight")
plt.close(fig)
